<center><font size=4 style="color:#BA4A00"><strong>购物类APP隐私政策获取</strong></font></center>

**在Kaggle中运行该代码时，请注意修改相应的文件路径。**

**本课程所用数据集均已上传为Kaggle公开数据集，地址如下：**

https://www.kaggle.com/datasets/jianzhangzhang/text-mining-course-data

### 任务要求

- 从应用市场中获取购物类别APP的隐私政策，保存在本地文本文件中；
- 应用市场采用**百度应用市场**；
- 每个APP的隐私政策保存在一个html文件中，命名格式为`APP名字.html`。

### 1. 分析应用市场页面源代码和返回数据

借助Chrome浏览器的页面`审查`功能，然后使用`requests`自动发送请求，分析返回数据。

In [1]:
import requests

In [2]:
# pn即page number
# boardId表示APP 类别
url = 'https://mobile.baidu.com/api/board?boardid=board_101_0316&boardId=34612&pn=0'

In [3]:
# 设置代理
proxies = {
	"http": None,
	"https": None,
}

In [4]:
# 发送请求
response = requests.get(url, proxies = proxies)

In [5]:
# 获取返回内容
cont = response.content

In [6]:
# 查看返回内容的数据类型
type(cont)

bytes

In [7]:
# 解码返回内容，使用UTF-8编码
cont_str = cont.decode('UTF-8')

In [8]:
type(cont_str)

str

In [9]:
# 将返回内容字符串转换为字典
cont_dict = eval(cont_str.replace('true','True'))

In [10]:
# 查看字典中的keys
cont_dict.keys()

dict_keys(['data', 'errorCode', 'errorMessage', 'errorObj'])

In [11]:
# 查看data的数据类型
type(cont_dict['data'])

dict

In [12]:
data_dict = cont_dict['data']

In [13]:
data_dict.keys()

dict_keys(['total', 'pn', 'data', 'hasNextPage', 'title'])

In [14]:
# 抽取data_dict中的字符信息
for item in data_dict.keys():
    exec('{} = data_dict["{}"]'.format(item,item)) # exec与eval的区别，https://blog.csdn.net/zhenaoxi1077/article/details/80903709

In [15]:
# 该类别（网络购物）下共有188个APP
total

188

In [16]:
# 现在是第几页
pn

0

In [17]:
# 还有没有下一页内容
hasNextPage

True

In [18]:
# 这个类别的名字是什么
title

'网上购物'

In [19]:
# 最关键的来了，包含的APP信息有哪些
type(data)

list

In [20]:
len(data)

14

In [21]:
data[1]

{'docid': 5001612384,
 'packageid': 1766075084,
 'package': 'com.taobao.taobao',
 'sname': '淘宝',
 'icon': 'https://gdown.baidu.com/appcenter/source/3117733246/6c6d31d260ed4e0a028fd4651714a490/res/mipmap-xhdpi-v4/ic_tb_app_foreground.png',
 'type': '0',
 'showSize': '107.56MB',
 'manual_brief': '淘宝，也太好逛了吧！',
 'strDownload': '4.4亿',
 'downloadUrl': 'https://m.gdown.baidu.com/c6f3b6740987390e7d3ec2c3bfb044ca138bb670c61656e4615bc4b058d0ef25973fc801d7bf7233726e2b05f125d04dffb7a31d33e16b570f3761e15b5fbf0d1a1d02938ea578e26b54bc267776d0bf53fcd8c74f848e57777897241aa79bb349c10e831759c316f73de2760a30b45ba1da5a3f16134b1a51bca1fda9869b5ef8fa2355a6bc79e69a815a40fa9f3dcf7574431ad8e82e8950b982c03ff1d0e3416270a8227cfe73a753da8fa5c2bfe98b29529971982ab8084e295ebf5f136cbf7f29d6328f8d9f82205d50d11414fc06c0afc4ac3f22e8',
 'hidownParam': {'k': '4T3La3t6TGjqAPLB12wPNgmJcmIEcoWSN5C3BJn_sK3RruzoZic3XksbBSToTNPEgYtR88WtvFNpzV5bc9FoztN5C_kfJLR5BuhFNP5tte4BMGz5f0mghAVDfFZ2QC9KxeBSdLZ6D5Ckxv80nxVATomNkzlSEOQfjuYsAr

In [22]:
# 获取隐私政策链接
privacy_url = data[0]['privacyUrl']

In [23]:
privacy_url

'http://m.pinduoduo.net/private_policy.html'

In [24]:
# 获取APP名字
app_name = data[0]['sname']

### 2. 获取并解析APP隐私政策文本

In [25]:
import re

In [26]:
from bs4 import BeautifulSoup

In [27]:
response = requests.get(privacy_url, proxies = proxies)

In [28]:
# 查看返回状态是否正常，200为正常
response.status_code

200

In [29]:
html = response.content.decode('UTF-8')

In [30]:
# 使用linux shell命令在当前目录新建保存隐私政策网页文件的文件夹
! mkdir app_privacy

mkdir: 无法创建目录"app_privacy": 文件已存在


In [31]:
# 将隐私政策以html格式保存在建立的文件夹中，供今后分析
with open('./app_privacy/{}.html'.format(app_name),'w+') as f:
    f.write(html)

In [32]:
! ls ./app_privacy/

58同城.html    京东.html    省钱快报.html  淘宝.html	一淘.html
阿里巴巴.html  拼多多.html  识货.html	   唯品会.html
多点.html      千牛.html    苏宁易购.html  小红书.html


### 3. 自动化批量获取和保存APP隐私政策

In [33]:
import time
from requests.adapters import SSLError

In [36]:
for record in data:
    privacy_url = record['privacyUrl']
    app_name = record['package']
    try:
        response = requests.get(privacy_url, proxies = proxies)
        time.sleep(1)
        if response.status_code == 200:
            html = response.content.decode('UTF-8')
            with open('./app_privacy/{}.html'.format(app_name),'w+') as f:
                f.write(html)
                print('Save the Privacy Policy of {}'.format(app_name))
    except (SSLError):
        time.sleep(3)
        continue

print('Done')

Save the Privacy Policy of com.xunmeng.pinduoduo
Save the Privacy Policy of com.taobao.taobao
Save the Privacy Policy of com.jingdong.app.mall
Save the Privacy Policy of com.wuba
Save the Privacy Policy of com.xingin.xhs
Save the Privacy Policy of com.achievo.vipshop
Save the Privacy Policy of com.suning.mobile.ebuy
Save the Privacy Policy of com.alibaba.wireless
Save the Privacy Policy of com.wm.dmall
Save the Privacy Policy of com.taobao.qianniu
Save the Privacy Policy of com.jzyd.coupon
Save the Privacy Policy of com.hupu.shihuo
Save the Privacy Policy of com.taobao.etao
Done


### 4. It's Your Turn

请参考上面代码将`网络购物`类别下全部APP的隐私政策以`html`格式保存到本地文件夹中。